<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/pablo_camarillo_add_spark_setup/examples/notebooks/lab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Password:
sudo: a password is required
zsh:1: command not found: apt-get
zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'spark-3.5.2-bin-hadoop3.tgz'

[notice] A new release of pip is available: 23.3 -> 24.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 6.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 9.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812371 sha256=bedaf3f1adc3da915ba4872a55453dd2abbfd0771c7fad90ddd21021f38f03fc
  Stored in directory: /Users/miguelmendez/Library/Caches/pip/wheels/9d/29/ee/3a756632ca3f0a6870933bac1c9db6e4af2c068f019aba0ee1
Successfully built pyspark

[notice] A new release of pip is available: 23.3 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 24.2
[notice] To update,

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .master("local[*]") \
  .appName("ITESO-2024-SparkIntroduction") \
  .config("spark.driver.bindAddress","localhost") \
  .config("spark.ui.port","4040") \
  .getOrCreate()

sc = spark.sparkContext

24/08/30 11:45:58 WARN Utils: Your hostname, Miguels-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.0.0.2 instead (on interface en0)
24/08/30 11:45:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/30 11:45:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Create an RDD with a list of sentences
sentences = sentences = [
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit.",
    "Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.",
    "Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur.",
    "Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
    "Curabitur pretium tincidunt lacus. Nulla gravida orci a odio.",
    "Nullam varius, turpis et commodo pharetra, est eros bibendum elit, nec malesuada elit elit vel lectus.",
    "Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium.",
    "Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit.",
    "Sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.",
    "Neque porro quisquam est qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit.",
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat."
]
sentences_rdd = sc.parallelize(sentences)

# Tokenize the sentences into words
words_rdd = sentences_rdd.flatMap(lambda line: line.split())

print(words_rdd.collect())

['Lorem', 'ipsum', 'dolor', 'sit', 'amet,', 'consectetur', 'adipiscing', 'elit.', 'Sed', 'do', 'eiusmod', 'tempor', 'incididunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliqua.', 'Ut', 'enim', 'ad', 'minim', 'veniam,', 'quis', 'nostrud', 'exercitation', 'ullamco', 'laboris', 'nisi', 'ut', 'aliquip', 'ex', 'ea', 'commodo', 'consequat.', 'Duis', 'aute', 'irure', 'dolor', 'in', 'reprehenderit', 'in', 'voluptate', 'velit', 'esse', 'cillum', 'dolore', 'eu', 'fugiat', 'nulla', 'pariatur.', 'Excepteur', 'sint', 'occaecat', 'cupidatat', 'non', 'proident,', 'sunt', 'in', 'culpa', 'qui', 'officia', 'deserunt', 'mollit', 'anim', 'id', 'est', 'laborum.', 'Curabitur', 'pretium', 'tincidunt', 'lacus.', 'Nulla', 'gravida', 'orci', 'a', 'odio.', 'Nullam', 'varius,', 'turpis', 'et', 'commodo', 'pharetra,', 'est', 'eros', 'bibendum', 'elit,', 'nec', 'malesuada', 'elit', 'elit', 'vel', 'lectus.', 'Sed', 'ut', 'perspiciatis', 'unde', 'omnis', 'iste', 'natus', 'error', 'sit', 'voluptatem', 'accusantium'

In [3]:
# Compute the frequency of each word
word_counts_rdd = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
print(word_counts_rdd.collect())

[('magna', 2), ('aliqua.', 2), ('veniam,', 2), ('quis', 2), ('nisi', 2), ('irure', 1), ('qui', 3), ('pretium', 1), ('tincidunt', 1), ('odio.', 1), ('turpis', 1), ('nec', 1), ('perspiciatis', 1), ('consectetur,', 1), ('ipsum', 3), ('laboris', 2), ('in', 3), ('eu', 1), ('cupidatat', 1), ('bibendum', 1), ('elit', 2), ('iste', 1), ('dolores', 1), ('Neque', 1), ('dolor', 4), ('adipiscing', 2), ('eiusmod', 2), ('Ut', 2), ('exercitation', 2), ('ullamco', 2), ('ea', 2), ('Duis', 1), ('aute', 1), ('velit', 1), ('esse', 1), ('id', 1), ('est', 3), ('vel', 1), ('unde', 1), ('aspernatur', 1), ('porro', 1), ('labore', 2), ('aliquip', 2), ('ex', 2), ('officia', 1), ('orci', 1), ('a', 1), ('laudantium.', 1), ('quia', 3), ('fugit.', 1), ('nesciunt.', 1), ('quisquam', 1), ('Lorem', 2), ('incididunt', 2), ('ut', 5), ('dolore', 3), ('enim', 3), ('cillum', 1), ('nulla', 1), ('deserunt', 1), ('anim', 1), ('Nulla', 1), ('omnis', 1), ('error', 1), ('eos', 1), ('ratione', 1), ('dolorem', 1), ('sit', 5), ('elit

In [4]:
# Find the most common word
most_common_word = word_counts_rdd.takeOrdered(1, key=lambda x: -x[1])
print("Most common word:", most_common_word)

Most common word: [('ut', 5)]


In [5]:
# Compute the average word length
total_length_rdd = words_rdd.map(lambda word: len(word)).reduce(lambda a, b: a + b)
total_words = words_rdd.count()
average_word_length = total_length_rdd / total_words if total_words > 0 else 0
print("Average word length:", average_word_length)

Average word length: 5.605555555555555


In [8]:
# Get the longest word in the dataset
longest_word = words_rdd.sortBy(lambda word: len(word), ascending=False).first()
print("Longest word:", longest_word)

Longest word: reprehenderit


In [9]:
# Get the shortes word in the dataset
shortest_word = words_rdd.sortBy(lambda word: len(word), ascending=True).first()
print("Shortest word:", shortest_word)

Shortest word: a


In [12]:
# Stop the SparkContext
sc.stop()